# Introduction

Welcome to this tutorial on using jupyter notebooks with MoveIt 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial.

In [1]:
import time

# generic ros libraries
import rclpy
from rclpy.logging import get_logger

# moveit python library
from moveit.core.robot_state import RobotState
from moveit.planning import (
    MoveItPy,
    MultiPipelinePlanRequestParameters,
)

from ament_index_python.packages import get_package_share_directory
from moveit_configs_utils import MoveItConfigsBuilder

def plan_and_execute(
    robot,
    planning_component,
    logger,
    single_plan_parameters=None,
    multi_plan_parameters=None,
    sleep_time=0.0,
):
    """Helper function to plan and execute a motion."""
    # plan to goal
    logger.info("Planning trajectory")
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        logger.info("Executing plan")
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        logger.error("Planning failed")

    time.sleep(sleep_time)

# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
        MoveItConfigsBuilder(
            robot_name="aubo_C3", package_name="aubo_description"
        )
        .robot_description_semantic(file_path=get_package_share_directory("aubo_description") 
            + "/srdf/aubo.srdf.xacro")
        .robot_description(file_path=get_package_share_directory("aubo_description") 
            + "/urdf/aubo.urdf.xacro"
            ,mappings={
                    "robot_ip": "127.0.0.1",
                    "aubo_type": "aubo_C3",
                    "use_fake_hardware": "true",
                    }
            )
        .planning_pipelines("ompl", ["ompl",  "pilz_industrial_motion_planner"])
        .moveit_cpp(
            file_path=get_package_share_directory("jupyter_python_tutorial")
            + "/config/jupyter_notebook_prototyping.yaml"
        )
        .planning_scene_monitor(publish_robot_description=True,publish_robot_description_semantic=True)
        .to_moveit_configs()
    ).to_dict()

## Setup

In [2]:
# initialise rclpy (only for logging purposes)
rclpy.init()
logger = get_logger("moveit_py.pose_goal")

# instantiate moveit_py instance and a planning component for the aubo_arm
aubo = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
aubo_arm = aubo.get_planning_component("aubo_arm")
logger.info("MoveItPy instance created")

[INFO] [1712907096.923375629] [moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1712907096.924207411] [moveit_cpp_initializer]: Initialize node parameters
[INFO] [1712907096.924225709] [moveit_cpp_initializer]: Initialize node and executor
[INFO] [1712907096.989780896] [moveit_cpp_initializer]: Spin separate thread
[INFO] [1712907097.031872860] [moveit_rdf_loader.rdf_loader]: Loaded robot model in 0.0419231 seconds
[INFO] [1712907097.031946644] [moveit_robot_model.robot_model]: Loading robot model 'aubo'...
[WARN] [1712907097.031967198] [moveit_robot_model.robot_model]: Skipping virtual joint 'virtual_joint' because its child frame 'aubo_base' does not match the URDF frame 'world'
[INFO] [1712907097.031971650] [moveit_robot_model.robot_model]: No root/virtual joint specified in SRDF. Assuming fixed joint
[INFO] [1712907097.059351379] [moveit_kinematics_base.kinematics_base]: Joint weights for group 'aubo_arm': 1 1 1 1 1 1
[INFO] [1712907097.080542126] [moveit.ros_planning_interface.m

True

## Motion Planning Example

In [3]:
###########################################################################
# Plan 1 - set states with predefined string
###########################################################################

# set plan start state to current state
aubo_arm.set_start_state_to_current_state()

# set pose goal using predefined state
aubo_arm.set_goal_state(configuration_name="ready")

# plan to goal
plan_and_execute(aubo, aubo_arm, logger, sleep_time=3.0)

[INFO] [1712907114.172732026] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1712907114.172801174] [moveit_py]: Parameter 'plan_request_params.planner_id' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1712907114.172867248] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1712907114.188957387] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'aubo_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[WARN] [1712907114.216207449] [moveit.ros_planning.planning_pipeline]: The planner plugin did not fill out the 'planner_id' field of the MotionPlanResponse. Setting it to the planner ID name of the MotionPlanRequest assuming that the planner plugin does warn you if it does not use the requested planner.
[INFO] [1712907114

In [4]:
 ###########################################################################
# Plan 2 - set goal state with RobotState object
###########################################################################

# instantiate a RobotState instance using the current robot model
robot_model = aubo.get_robot_model()
robot_state = RobotState(robot_model)

# randomize the robot state
robot_state.set_to_random_positions()

# set plan start state to current state
aubo_arm.set_start_state_to_current_state()

# set goal state to the initialized robot state
logger.info("Set goal state to the initialized robot state")
aubo_arm.set_goal_state(robot_state=robot_state)

# plan to goal
plan_and_execute(aubo, aubo_arm, logger, sleep_time=3.0)

[INFO] [1712907156.356900987] [moveit_py.pose_goal]: Set goal state to the initialized robot state
[INFO] [1712907156.371771018] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1712907156.371830389] [moveit_py]: Parameter 'plan_request_params.planner_id' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1712907156.371887004] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1712907156.372441501] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'aubo_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1712907156.388080011] [moveit_kinematic_constraints.kinematic_constraints]: Constraint satisfied:: Joint name: 'shoulder_joint', actual value: -5.384287, desired value: -5.384287, tolerance_above: 0.000000, tolerance

In [5]:
###########################################################################
# Plan 3 - set goal state with PoseStamped message
###########################################################################
aubo_arm.set_start_state_to_current_state()

from geometry_msgs.msg import PoseStamped,Quaternion

pose_goal = PoseStamped()
quat = Quaternion(x=-0.38,y=0.0,z=0.653,w=0.653)
pose_goal.header.frame_id = "aubo_base"
pose_goal.pose.orientation = quat
pose_goal.pose.position.x = -0.2
pose_goal.pose.position.y = 0.2
pose_goal.pose.position.z = 0.3
aubo_arm.set_goal_state(pose_stamped_msg = pose_goal, pose_link = "wrist3_Link")

# plan to goal
plan_and_execute(aubo, aubo_arm, logger, sleep_time=3.0)

[INFO] [1712907311.226652507] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1712907311.226799397] [moveit_py]: Parameter 'plan_request_params.planner_id' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1712907311.226947073] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1712907311.227531217] [moveit_kinematic_constraints.kinematic_constraints]: Orientation constraint for link 'wrist3_Link' is probably incorrect: -0.380000, 0.000000, 0.653000, 0.653000. Assuming identity instead.
[WARN] [1712907311.227578581] [moveit_kinematic_constraints.kinematic_constraints]: Orientation constraint for link 'wrist3_Link' is probably incorrect: -0.380000, 0.000000, 0.653000, 0.653000. Assuming identity instead.
[INFO] [1712907311.227789272] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'aubo_arm' will

In [6]:
###########################################################################
# Plan 4 - set goal state with constraints
###########################################################################

# set plan start state to current state
aubo_arm.set_start_state_to_current_state()

# set constraints message
from moveit.core.kinematic_constraints import construct_joint_constraint

joint_values = {
    "shoulder_joint": 0.0,
    "upperArm_joint": -0.26,
    "foreArm_joint": 1.74,
    "wrist1_joint": 0.43,
    "wrist2_joint": 1.57,
    "wrist3_joint": 0.0,
}
robot_state.joint_positions = joint_values
joint_constraint = construct_joint_constraint(
    robot_state=robot_state,
    joint_model_group=aubo.get_robot_model().get_joint_model_group("aubo_arm"),
)
aubo_arm.set_goal_state(motion_plan_constraints=[joint_constraint])

# plan to goal
plan_and_execute(aubo, aubo_arm, logger, sleep_time=3.0)


[INFO] [1712907501.479072757] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1712907501.479130743] [moveit_py]: Parameter 'plan_request_params.planner_id' not found in config use default value instead, check parameter type and namespace in YAML file
[WARN] [1712907501.479162426] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1712907501.479581581] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'aubo_arm' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[WARN] [1712907501.493866965] [moveit.ros_planning.planning_pipeline]: The planner plugin did not fill out the 'planner_id' field of the MotionPlanResponse. Setting it to the planner ID name of the MotionPlanRequest assuming that the planner plugin does warn you if it does not use the requested planner.
[INFO] [1712907501

In [7]:
###########################################################################
# Plan 5 - Planning with Multiple Pipelines simultaneously
###########################################################################

# set plan start state to current state
aubo_arm.set_start_state_to_current_state()

# set pose goal with PoseStamped message
aubo_arm.set_goal_state(configuration_name="ready")

# initialise multi-pipeline plan request parameters

multi_pipeline_plan_request_params = MultiPipelinePlanRequestParameters(
    aubo, ["ompl_rrtc", "pilz_lin"]
)

# plan to goal
plan_and_execute(
    aubo,
    aubo_arm,
    logger,
    multi_plan_parameters=multi_pipeline_plan_request_params,
    sleep_time=3.0,
)

[INFO] [1712907616.723271035] [moveit_py.pose_goal]: Planning trajectory
[INFO] [1712907616.742200003] [moveit.pilz_industrial_motion_planner.trajectory_generator_ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1712907616.742263982] [moveit.ompl_planning.model_based_planning_context]: Planner configuration 'aubo_arm[RRTConnectkConfigDefault]' will use planner 'geometric::RRTConnect'. Additional configuration parameters will be set when the planner is constructed.
[INFO] [1712907616.743125417] [moveit.pilz_industrial_motion_planner.trajectory_generator]: Generating PTP trajectory...
[WARN] [1712907616.755216995] [moveit.ros_planning.planning_pipeline]: The planner plugin did not fill out the 'planner_id' field of the MotionPlanResponse. Setting it to the planner ID name of the MotionPlanRequest assuming that the planner plugin does warn you if it does not use the requested planner.
[INFO] [1712907616.774973620] [moveit_py.pose_goal]: Executing plan
[INFO] [1712907616.7750